In [ ]:
from eose.instruments import  CircularGeometry, RectangularGeometry, BasicSensor

from pydantic import ValidationError

# Example usage of SphericalGeometry class used to define the field of view of the instrument

circ_geom = CircularGeometry(diameter=90.0)
print(circ_geom)

rect_geom = RectangularGeometry(angle_height= 90.0, angle_width= 45)
print(rect_geom)


In [ ]:
# Example simple usage of eose-api BasicSensor class

sensor = BasicSensor(   mass= 100.5,
                        volume= 0.75,
                        power= 150.0,
                        field_of_view = CircularGeometry(diameter=30.0),
                        data_rate= 10.5,
                        bits_per_pixel= 16
                    )

print(sensor)

sensor = BasicSensor(   mass= 100.5,
                        volume= 0.75,
                        power= 150.0,
                        orientation= list([0.5,0.5,0.5,0.5]),
                        field_of_view = RectangularGeometry(ngle_width=30.0, angle_height=10.0),
                        data_rate= 50.5,
                        bits_per_pixel= 8
                    )

print(sensor)

sensor = BasicSensor(   mass= 100.5,
                    volume= 0.75,
                    power= 150.0,
                    orientation= list([0.5,0.5,0.5,0.5]),
                    data_rate= 50.5,
                    bits_per_pixel= 8
                )

print(sensor)

display(sensor.model_dump_json())

In [ ]:
%reset -f

## Example usage with TAT-C and InstruPy

In [ ]:
import json
from datetime import datetime, timedelta, timezone

from instrupy.basic_sensor_model import BasicSensorModel

from tatc.schemas import Instrument as TATC_Instrument, Satellite as TATC_Satellite, TwoLineElements, Point
from tatc.analysis import collect_orbit_track, OrbitCoordinate, OrbitOutput
from tatc.analysis import (
    collect_multi_observations,
    aggregate_observations,
    reduce_observations,
)

from instrupy.basic_sensor_model import BasicSensorModel as InstruPy_BasicSensorModel

from eose.instruments import CircularGeometry, BasicSensor
from eose.datametric import DataMetricRequest, BasicSensorDataMetricResponse
from eose.propagation import PropagationRecord, PropagationRequest, PropagationResponse
from eose.orbits import GeneralPerturbationsOrbitState
from eose.utils import CartesianReferenceFrame, FixedOrientation
from eose.coverage import (
    CoverageRecord,
    CoverageRequest,
    CoverageResponse,
    CoverageSample,
)
from eose.grids import UniformAngularGrid
from eose.satellites import Satellite

from shapely.geometry import box, mapping
from joblib import Parallel, delayed
from scipy.stats import hmean
import pandas as pd

from astropy.time import Time as AstroPy_Time



In [ ]:
# define the orbit and the instrument
iss_omm_str = '[{"OBJECT_NAME":"ISS (ZARYA)","OBJECT_ID":"1998-067A","EPOCH":"2024-06-07T09:53:34.728000","MEAN_MOTION":15.50975122,"ECCENTRICITY":0.0005669,"INCLINATION":51.6419,"RA_OF_ASC_NODE":3.7199,"ARG_OF_PERICENTER":284.672,"MEAN_ANOMALY":139.0837,"EPHEMERIS_TYPE":0,"CLASSIFICATION_TYPE":"U","NORAD_CAT_ID":25544,"ELEMENT_SET_NO":999,"REV_AT_EPOCH":45703,"BSTAR":0.00033759,"MEAN_MOTION_DOT":0.00019541,"MEAN_MOTION_DDOT":0}]'
iss_omm = json.loads(iss_omm_str)[0]

basic_sensor = BasicSensor(   mass= 100.5,
                        volume= 0.75,
                        power= 150.0,
                        field_of_view = CircularGeometry(diameter=60.0),
                        data_rate= 10.5,
                        bits_per_pixel= 16
                    )

mission_start = datetime(2024, 1, 1, tzinfo=timezone.utc)
mission_duration = timedelta(days=1)

In [ ]:
# Run propagation with TAT-C to get satellite states
def propagate_tatc(request: PropagationRequest) -> PropagationResponse:
    return PropagationResponse(
        records=collect_orbit_track(
            TATC_Satellite(
                name=request.orbit.object_name,
                orbit=TwoLineElements(tle=request.orbit.to_tle()),
            ),
            TATC_Instrument(name="Instrument"),
            pd.date_range(
                request.start, request.start + request.duration, freq=request.time_step
            ),
            coordinates=(
                OrbitCoordinate.ECI
                if request.frame == CartesianReferenceFrame.ICRF
                else OrbitCoordinate.ECEF
            ),
            orbit_output=OrbitOutput.POSITION_VELOCITY,
        ).apply(
            lambda r: PropagationRecord(
                time=r.time,
                frame=request.frame,
                position=r.geometry.coords[0],
                velocity=r.velocity.coords[0],
                body_orientation=FixedOrientation.NADIR_GEOCENTRIC,
            ),
            axis=1,
        ),
    )



request = PropagationRequest(
    orbit=GeneralPerturbationsOrbitState.from_omm(iss_omm),
    start=mission_start,
    duration=mission_duration,
    time_step=timedelta(minutes=0.25),
    frame=CartesianReferenceFrame.ICRF,
)

display(request.model_dump_json())

prop_response = propagate_tatc(request)

display(prop_response.model_dump_json())

prop_data = prop_response.as_dataframe()

display(prop_data)


In [ ]:
# Run coverage analysis with TAT-C to get access-times at Target ground points. It is assumed that the sensor shall be of Circular FOV geometry.


def coverage_tatc(request: CoverageRequest) -> CoverageResponse:
    unique_ids = len(set(target.id for target in request.targets)) == len(
        request.targets
    )
    points = [
        Point(
            id=target.id if unique_ids and isinstance(target.id, int) else i,
            longitude=target.position[0],
            latitude=target.position[1],
            altitude=target.position[2] if len(target.position) > 2 else 0,
        )
        for i, target in enumerate(request.targets)
    ]
    satellites = [
        TATC_Satellite(
            name=satellite.orbit.object_name,
            orbit=TwoLineElements(tle=satellite.orbit.to_tle()),
            instruments=[
                TATC_Instrument(name="Default", field_of_regard=satellite.field_of_view)
            ],
        )
        for satellite in request.satellites
    ]

    aggregated_obs = aggregate_observations(
        pd.concat(
            Parallel(-1)(
                delayed(collect_multi_observations)(
                    point,
                    satellites,
                    request.start,
                    request.start + request.duration,
                )
                for point in points
            )
        )
    )
    reduced_obs = reduce_observations(aggregated_obs)

    records = list(
        reduced_obs.apply(
            lambda r: CoverageRecord(
                target=request.targets[
                    points.index(next(p for p in points if p.id == r["point_id"]))
                ],
                samples=aggregated_obs[aggregated_obs.point_id == r["point_id"]].apply(
                    lambda s: CoverageSample(start=s.start, duration=s.end - s.start),
                    axis=1,
                ),
                mean_revisit=(
                    None
                    if pd.isnull(r["revisit"])
                    else timedelta(seconds=r["revisit"].total_seconds())
                ),
                number_samples=r["samples"],
            ),
            axis=1,
        )
    ) + [
        CoverageRecord(target=request.targets[i], mean_revisit=None, number_samples=0)
        for i, point in enumerate(points)
        if not any(reduced_obs["point_id"] == point.id)
    ]
    records.sort(key=lambda r: r.target.id)

    return CoverageResponse(
        records=records,
        harmonic_mean_revisit=(
            None
            if reduced_obs.dropna(subset="revisit").empty
            else timedelta(
                seconds=hmean(
                    reduced_obs.dropna(subset="revisit")["revisit"].dt.total_seconds()
                )
            )
        ),
        coverage_fraction=len(reduced_obs.index) / len(points),
    )



request = CoverageRequest(
    satellites=[
        Satellite(
            orbit=GeneralPerturbationsOrbitState.from_omm(iss_omm), field_of_view=basic_sensor.field_of_view.diameter
        )
    ],
    targets=UniformAngularGrid(
        delta_latitude=20, delta_longitude=20, region=mapping(box(-180, -50, 180, 50))
    ).as_targets(),
    start=mission_start,
    duration=mission_duration,
)

display(request.model_dump_json())

coverage_response = coverage_tatc(request)

display(coverage_response.model_dump_json())

coverage_data = coverage_response.as_dataframe()

display(coverage_data)

In [ ]:
# define the InstruPy basic sensor model
instrupy_basic_sensor = InstruPy_BasicSensorModel.from_dict({"orientation": {"referenceFrame": "SC_BODY_FIXED", "convention": "REF_FRAME_ALIGNED"},
                                         "fieldOfViewGeometry": {"shape": "CIRCULAR", "diameter": basic_sensor.field_of_view.diameter}
                                                            })

def propagation_records_within_time_range(propagate_response, start_time, duration):

    end_time = start_time + duration
    #print("Start time and end date are: ", start_time, end_time)
    # iterate through all the records, and store the records corresponding to the given time range
    filtered_propagate_records = []
    for record in propagate_response.records:
                
        # Check if the time falls within the range
        if start_time <= record.time <= end_time:
            filtered_propagate_records.append(record)
            #print(record.time)

    return filtered_propagate_records

# iterate through the coverage records
for record in coverage_response.records:
    # each record consists of coverage samples (start, duration) for a target point
    #print(record)
    #print("\n")
    # iterate through each coverage sample within a coverage record
    target = record.target
    print("target = ", target)
    for sample in record.samples:
        print(sample.start, sample.duration, "\n")

        # from the propagation response, find satellite states within the coverage sample period
        pr = propagation_records_within_time_range(prop_response, sample.start, sample.duration)

        print("Propagate state: ", pr)

        if pr is not []: # if propagation states are available within the time-period
            # iterate through the propagation states
            for _pr in pr: 
                # get the time in Julian Date UT1
                time_utc = AstroPy_Time(_pr.time, scale='utc') # Convert to astropy Time object
                time_ut1 = time_utc.ut1  # Convert to UT1 time scale
                jd_ut1 = time_ut1.jd  # Get the Julian Date
        
                # calculate the corresponding instantaneous data metrics
                SpacecraftOrbitState = {'time [JDUT1]':jd_ut1, 'x [km]': _pr.position[0]*1e-3, 'y [km]': _pr.position[1]*1e-3, 'z [km]': _pr.position[2]*1e-3, 
                                        'vx [km/s]': _pr.velocity[0]*1e-3, 'vy [km/s]': _pr.velocity[1]*1e-3, 'vz [km/s]': _pr.velocity[2]*1e-3}
                TargetCoords = {'lat [deg]': target.position[1], 'lon [deg]': target.position[0]}
                obsv_metrics = instrupy_basic_sensor.calc_data_metrics(SpacecraftOrbitState, TargetCoords)
                print(obsv_metrics)
        
        
        
    

In [ ]:
# Run InstruPy Basic Sensor datametrics calculator to get the data-metricss


def datametrics_instrupy(request: DataMetricRequest) -> BasicSensorDataMetricResponse:
    
    
    # Filter propagation record for each access and compute metrics throughout the access period at each propagation time-instant

    # iterate through each access
    for index, access in enumerate(coverage_data):
        print(access.records)
    


    return